In [1]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, IBMQ
from qiskit.circuit.library import QFT
from qiskit.circuit.library.standard_gates import PhaseGate
from qiskit import Aer
import numpy as np

backend = Aer.get_backend('statevector_simulator')
   

In [6]:

def qmult(x,y):

    '''Creating binary representations of the decimal inputs'''
    xbin=bin(x)[2:]
    ybin=bin(y)[2:]
    n=max(len(xbin),len(ybin))
    
    '''Creatong registers for input1, input2, product and readout'''
    xreg = QuantumRegister(n, name="xreg")
    yreg = QuantumRegister(n, name="yreg")
    preg = QuantumRegister(2*n, name="preg")
    c = ClassicalRegister(2*n,'c')

    qc = QuantumCircuit()
    qc.add_register(xreg,yreg,preg,c)

    #Encoding
    pos=0
    for i in "".join(reversed(xbin)):
        if i=='1':
            qc.x(xreg[pos])
        pos+=1
    
    pos=0
    for i in "".join(reversed(ybin)):
        if i=='1':
            qc.x(yreg[pos])
        pos+=1

    # build multiplication circuit

    qc.append(QFT(2*n, do_swaps=False).to_gate(), preg[:])

    for j in range(n):
        for i in range(n):
            for k in range(2*n):
                phase = (2 * np.pi) / (2 ** (i + j + k + 3 - 2 * n))
                qc.append(PhaseGate(phase).control(2),[xreg[n - 1 - j], yreg[n - 1 - i], preg[k]],)

    qc.append(QFT(2*n, do_swaps=False).inverse().to_gate(), preg[:])

    for i in range(2*n):
        qc.measure(preg[i],c[i])
 

    print(qc)

    job = execute(qc, backend, shots=2000)
    result = job.result()
    counts = result.get_counts()
    ans=list(counts.keys())
    p=0
    a=0
    for i in "".join(reversed(ans[0])):
        p+=int(i)*(2**a)
        a+=1
    
    return p

In [7]:
qmult(10,2)

                                                                          »
xreg_0: ──────────────────────────────────────────────────────────────────»
         ┌───┐                                                            »
xreg_1: ─┤ X ├────────────────────────────────────────────────────────────»
         └───┘                                                            »
xreg_2: ──────────────────────────────────────────────────────────────────»
         ┌───┐                                                            »
xreg_3: ─┤ X ├───■────────■────────■────────■───────■───────■───────■─────»
         └───┘   │        │        │        │       │       │       │     »
yreg_0: ─────────┼────────┼────────┼────────┼───────┼───────┼───────┼─────»
         ┌───┐   │        │        │        │       │       │       │     »
yreg_1: ─┤ X ├───┼────────┼────────┼────────┼───────┼───────┼───────┼─────»
         └───┘   │        │        │        │       │       │       │     »
yreg_2: ────

/opt/homebrew/anaconda3/envs/Quantum/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/anaconda3/envs/Quantum/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


20